In [1]:
import pandas as pd
from datetime import datetime
from pathlib import Path
import re
import unicodedata
import requests
import json


In [2]:
SPREADSHEET_ID = "1IPS5dBSGtwYVbjsfbaMCYIWnOuRmJcbequohNxCyGVw"
GID = "1625408792" 
csv_url = f"https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/export?format=csv&gid={GID}"

df_raw = pd.read_csv(csv_url)

In [3]:
df_raw.to_csv("../data/raw/data_raw.csv", index=False)

In [4]:
df_raw.columns

Index(['Timestamp', 'How old are you?', 'What industry do you work in?',
       'Job title',
       'If your job title needs additional context, please clarify here:',
       'What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)',
       'How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.',
       'Please indicate the currency',
       'If "Other," please indicate the currency here: ',
       'If your income needs additional context, please provide it here:',
       'What country do you work in?',
       'If you're in the U.S., what state do you work in?',
       'What city do you work in?',
       'How many years of professional work experience do you have overall?',
       

In [5]:
short_names = [
    "timestamp",
    "age",
    "industry",
    "job_title",
    "job_title_context",
    "annual_salary_raw",
    "additional_compensation_raw",
    "currency",
    "currency_other",
    "income_context",
    "country_raw",
    "state_us_raw",
    "city_raw",
    "experience_years_total",
    "experience_years_field",
    "education_level",
    "gender",
    "race"
]

In [6]:
rename_dict = dict(zip(df_raw.columns.tolist(), short_names))
df = df_raw.rename(columns=rename_dict)
df.columns

Index(['timestamp', 'age', 'industry', 'job_title', 'job_title_context',
       'annual_salary_raw', 'additional_compensation_raw', 'currency',
       'currency_other', 'income_context', 'country_raw', 'state_us_raw',
       'city_raw', 'experience_years_total', 'experience_years_field',
       'education_level', 'gender', 'race'],
      dtype='object')

In [7]:
df.head()


,timestamp,age,industry,job_title,job_title_context,annual_salary_raw,additional_compensation_raw,currency,currency_other,income_context,country_raw,state_us_raw,city_raw,experience_years_total,experience_years_field,education_level,gender,race
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,"62,000",3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,"60,000",7000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White


In [8]:
data_dictionary_v1 = pd.DataFrame({
    "original_name": df_raw.columns,
    "new_name": df.columns,
    "input_type": df.dtypes.values
})

descripcion_es = {
    "timestamp": "Fecha y hora en que el encuestado respondió el formulario.",
    "age": "Rango de edad del encuestado.",
    "industry": "Industria en la que trabaja el encuestado.",
    "job_title": "Cargo o título profesional reportado.",
    "job_title_context": "Texto libre con contexto adicional sobre el cargo.",
    "annual_salary_raw": "Salario anual reportado (texto/numérico sin estandarizar).",
    "additional_compensation_raw": "Compensación monetaria adicional anual reportada (sin estandarizar).",
    "currency": "Moneda seleccionada para reportar salario.",
    "currency_other": "Texto libre cuando la moneda es 'Other'.",
    "income_context": "Texto libre con contexto adicional sobre ingresos.",
    "country_raw": "País reportado (sin estandarizar).",
    "state_us_raw": "Estado en EE.UU. reportado (sin estandarizar).",
    "city_raw": "Ciudad reportada (sin estandarizar).",
    "experience_years_total": "Años totales de experiencia profesional (rango).",
    "experience_years_field": "Años de experiencia en el campo (rango).",
    "education_level": "Nivel educativo más alto completado.",
    "gender": "Género reportado.",
    "race": "Raza reportada (puede incluir múltiples categorías)."
}

data_dictionary_v1["descripcion_es"] = data_dictionary_v1["new_name"].map(descripcion_es)

input_field_type = {
    "timestamp": "defect",
    "age": "predefined_range",
    "industry": "select_with_other_option",
    "job_title": "open_text",
    "job_title_context": "open_text",
    "annual_salary_raw": "open_text_number",
    "additional_compensation_raw": "open_text_number",
    "currency": "predefined_range",
    "currency_other": "open_text",
    "income_context": "open_text",
    "country_raw": "open_text",
    "state_us_raw": "predefined_range",
    "city_raw": "open_text",
    "experience_years_total": "predefined_range",
    "experience_years_field": "predefined_range",
    "education_level": "predefined_range",
    "gender": "predefined_range",
    "race": "predefined_range"
}

data_dictionary_v1["input_field_type"] = data_dictionary_v1["new_name"].map(input_field_type)

is_required = {
    "timestamp": True,
    "age": True,
    "industry": False,
    "job_title": True,
    "job_title_context": False,
    "annual_salary_raw": True,
    "additional_compensation_raw": False,
    "currency": True,
    "currency_other": False,
    "income_context": False,
    "country_raw": True,
    "state_us_raw": False,
    "city_raw": True,
    "experience_years_total": True,
    "experience_years_field": True,
    "education_level": False,
    "gender": False,
    "race": False
}

data_dictionary_v1["is_required"] = data_dictionary_v1["new_name"].map(is_required)

data_dictionary_v1 = data_dictionary_v1[[
    "original_name",
    "new_name",
    "descripcion_es",
    "input_type",
    "input_field_type",
    "is_required"
]]

In [9]:
data_dictionary_v1.to_excel("../docs/data_dictionary_v1.xlsx", index=False)
data_dictionary_v1.to_csv("../docs/data_dictionary_v1.csv", index=False)

data_dictionary_v1.head()

,original_name,new_name,descripcion_es,input_type,input_field_type,is_required
0,Timestamp,timestamp,Fecha y hora en que el encuestado respondió el...,object,defect,True
1,How old are you?,age,Rango de edad del encuestado.,object,predefined_range,True
2,What industry do you work in?,industry,Industria en la que trabaja el encuestado.,object,select_with_other_option,False
3,Job title,job_title,Cargo o título profesional reportado.,object,open_text,True
4,"If your job title needs additional context, pl...",job_title_context,Texto libre con contexto adicional sobre el ca...,object,open_text,False


In [10]:
# Función para normalizar texto: minúsculas, sin acentos, sin caracteres especiales, sin espacios extra
def normalize_text(x):
    if pd.isna(x):
        return None
    x = str(x).strip().lower()
    x = unicodedata.normalize("NFKD", x).encode("ascii", "ignore").decode("ascii")
    x = re.sub(r"[^\w\s]", " ", x)  
    x = re.sub(r"\s+", " ", x).strip()
    return x

In [11]:
# Aplicar normalización 
df["country_norm"] = df["country_raw"].apply(normalize_text)
df["city_norm"] = df["city_raw"].apply(normalize_text)
df["job_title_norm"] = df["job_title"].apply(normalize_text)

In [12]:
# Crear columna de país limpio con base en la columna normalizada
df["country_clean"] = df["country_norm"]

# Definir y remplazar variantes para USA y UK
usa_variants = {
    "u s", "u s a", "u s a", "u.s", "u.s.", "u.s.a", "u.s.a.", "us", "usa",
    "united states", "united states of america", "the united states", "the us",
    "america", "unitedstates",
    "united statss", "uniteed states", "uniited states", "unite states",
    "unites states", "united stares", "united state", "united stated", "united stateds"
}

uk_variants = {
    "united kingdom", "uk", "u k", "u.k", "u.k.", "great britain", "britain"
}

df.loc[df["country_norm"].isin(usa_variants), "country_clean"] = "USA"
df.loc[df["country_norm"].isin(uk_variants), "country_clean"] = "UK"

In [13]:
def apply_country_patterns(df, norm_column="country_norm", out_column="country_clean"):

    pattern_rules = [
        # USA (incluye typos: untied, unitied, sttes, statues, etc.)
        ("USA",r"\busa\b|\bu\.?s\.?a\.?\b|\bu\.?s\.?\b|\bamerica\b|\bthe us\b|\bthe united states\b|\bunited states\b|"
         r"\buni\w*\s+st\w*\b|\buntied\s+states\b"),
        ("UK",r"\bunited kingdom\b|\buk\b|\bu\.?k\.?\b|great britain|britain|\bengland\b|\bscotland\b|\bwales\b|\bnorthern ireland\b|englang|\bkingd\w*\b"),
        ("Ireland", r"\bireland\b"),
        ("Canada", r"\bcanada\b|\bcan\b|\bcanad\b|\bcanad\w*\b|\bcsnada\b"), 
        ("Australia", r"\baustralia\b|\baustral"),
        ("Germany", r"\bgermany\b|\bgerman(y)?\b"),
        ("France", r"\bfrance\b"),
        ("Spain", r"\bspain\b|\bespana\b"),
        ("China", r"\bchina\b|mainland china"),
        ("Japan", r"\bjapan\b"),
        ("Denmark", r"\bdenmark\b|\bdanmark\b"),
        ("Philippines", r"\bphilippines\b|\bphil\b"),
        ("Austria", r"\baustria\b|\baustri"),
        ("Argentina", r"\bargentina\b|\bargentin"),
        ("Romania", r"\bromania\b|\bromani"),
        ("Thailand", r"\bthailand\b"),
        ("UAE", r"\buae\b|\bunited arab emirates\b|\bu\.?a\.?e\.?\b"),
        ("New Zealand", r"\bnew zealand\b|\baotearoa\b"),
        ("Netherlands", r"\bnetherlands\b|\bnederland\b|\bholland\b|\bnl\b"),
    ]

    # Si no usamos la limpieza manual, aseguramos que la columna de salida exista 
    if out_column not in df.columns:
        df[out_column] = df[norm_column]

    # Aplicar reglas
    for country, pattern in pattern_rules:
        mask = df[norm_column].str.contains(pattern, na=False, regex=True)
        df.loc[mask, out_column] = country

    # Title Case para los que no fueron mapeados a estos países
    main_countries = {c for c, _ in pattern_rules}

    mask_other = df[out_column].notna() & ~df[out_column].isin(main_countries)
    df.loc[mask_other, out_column] = (
        df.loc[mask_other, out_column]
          .astype(str)
          .str.strip()
          .str.title()
    )

    return df

df = apply_country_patterns(df, norm_column="country_norm", out_column="country_clean")


/var/folders/0s/2_dbpwwx125093whj8n7wyf00000gn/T/ipykernel_27717/3251682833.py:33: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df[norm_column].str.contains(pattern, na=False, regex=True)


In [14]:
mask_no_letters = ~df["country_clean"].astype(str).str.contains(r"[a-zA-Z]", na=False)
df.loc[mask_no_letters, ["country_raw","country_norm", "country_clean"]]

,country_raw,country_norm,country_clean
19747,🇺🇸,,
22406,🇺🇸,,
24017,🇺🇸,,
26613,🇺🇸,,
28177,1,1,1


In [15]:
def clean_unknown_countries(
    df,
    column="country_clean",
    max_words=4,
    extra_noise_words=None
):
    df = df.copy()

# No contiene letras
    mask_no_letters = ~df[column].astype(str).str.contains(r"[a-zA-Z]", na=False)
    df.loc[mask_no_letters, column] = "Unknown"

# Demasiadas palabras
    mask_long = (
        df[column].notna() &
        (df[column].astype(str).str.split().str.len() > max_words)
    )
    df.loc[mask_long, column] = "Unknown"

# Palabras de ruido
    default_noise_words = [
        "raise", "bonus", "salary", "commission",
        "benefits", "quota", "remote", "year",
        "deducted"
    ]

# Agregar palabras de ruido adicionales si se proporcionan en el argumento de la función
    if extra_noise_words:
        default_noise_words.extend(extra_noise_words)

    pattern_noise = "|".join(default_noise_words)

    mask_noise = df[column].astype(str).str.contains(
        pattern_noise,
        case=False,
        na=False,
        regex=True
    )

    df.loc[mask_noise, column] = "Unknown"

    return df


In [16]:
df = clean_unknown_countries(
    df,
    column="country_clean",
    max_words=4,
    extra_noise_words=["work", "job", "company", "international"]
)

In [17]:
df["country_clean"].value_counts().head(10)

country_clean
USA            23242
Canada          1691
UK              1595
Australia        392
Germany          199
Ireland          130
New Zealand      126
Netherlands       91
France            69
Spain             50
Name: count, dtype: int64

In [18]:
df["annual_salary"] = (
    df["annual_salary_raw"]
        .astype(str)
        .str.replace(",", "", regex=False) 
)

df["annual_salary"] = pd.to_numeric(df["annual_salary"], errors="coerce").astype("Int64")

In [19]:
df["currency"].value_counts()

currency
USD        23473
CAD         1676
GBP         1594
EUR          654
AUD/NZD      505
Other        171
CHF           37
SEK           37
JPY           23
ZAR           17
HKD            4
Name: count, dtype: int64

In [20]:
# Listas de monedas válidas revisando la data
VALID_CURRENCIES = {
    "USD","CAD","GBP","EUR","AUD","NZD","CHF","SEK","JPY","ZAR","HKD",
    "INR","SGD","NOK","DKK","MYR","BRL","PLN","CZK","ILS","PHP","ARS",
    "CNY","MXN","TRY","BDT","TTD","ZMW",
    "KRW","THB","IDR","LKR","SAR","NGN","COP","TZS","HRK"
}

CURRENCY_NAME_KEYWORDS = {
    # Philippines
    "PHILIPPINE": "PHP",
    "PESO (PHP)": "PHP",
    "PESOS (PHP)": "PHP",
    # Korea
    "KOREAN WON": "KRW",
    "WON": "KRW",
    # Thailand
    "THAI BAHT": "THB",
    "BAHT": "THB",
    # Indonesia / Sri Lanka
    "INDONES": "IDR",
    "RUPIAH": "IDR",
    "SRI LANK": "LKR",
    # Poland
    "POLISH": "PLN",
    "ZLOTY": "PLN",
    # Denmark / Norway / Sweden
    "DANISH": "DKK",
    "KRONER": "DKK",
    "NORWEG": "NOK",
    "KORONA": "SEK",
    # Europe
    "EURO": "EUR",
    # Argentina / Mexico / Colombia
    "ARGENT": "ARS",
    "MEXIC": "MXN",
    "COLOMB": "COP",
    # China
    "RMB": "CNY",
    "YUAN": "CNY",
    # Israel
    "SHEKEL": "ILS",
    "ISRAEL": "ILS",
    # US / Dollars
    "AMERICAN DOLLAR": "USD",
    "US DOLLAR": "USD",
    "U S DOLLAR": "USD",
    "DOLLAR": "USD", 
    # Croatia 
    "CROATIAN KUNA": "HRK",
    "KUNA": "HRK",
    # India
    "INDIAN RUPEE": "INR",
    "INDIAN RUPEES": "INR",
    "RUPEE": "INR",
    "RUPEES": "INR",
    # Brazil
    "BR$": "BRL",
    "BRAZIL": "BRL",
    # Czech
    "CZECH": "CZK",
    "CROWNS": "CZK",
    "KORUNA": "CZK",
    # Malaysia
    "RM": "MYR",
    "RINGGIT": "MYR",
    # Taiwan
    "NTD": "TWD",
    "TAIWAN": "TWD",    
}

TOKEN_NORMALIZATION = {
    "NIS": "ILS",
    "RM": "MYR",   
    "NTD": "TWD", 
}


# Función para resolver currency_other
def resolve_currency_other(x):
    
    if pd.isna(x):
        return None

    text = str(x).strip().upper()

    if "EQUITY" in text:
        return None

    tokens = re.findall(r"\b[A-Z]{3}\b", text)
    for t in tokens:
        t_norm = TOKEN_NORMALIZATION.get(t, t)
        if t_norm in VALID_CURRENCIES:
            return t_norm

    for key, iso in CURRENCY_NAME_KEYWORDS.items():
        if key in text:
            return iso if iso in VALID_CURRENCIES else iso 

    return None

# Función principal para estandarizar monedas
def standardize_currencies(
    df,
    currency_col="currency",
    currency_other_col="currency_other",
    out_col="currency_final",
    out_other_clean_col="currency_other_clean"
):
    df = df.copy()

    # Normalize main currency column
    df[out_col] = df[currency_col].astype(str).str.strip().str.upper()
    df.loc[df[currency_col].isna(), out_col] = None

    # AUD/NZD -> AUD
    df.loc[df[out_col].eq("AUD/NZD"), out_col] = "AUD"

    df[out_other_clean_col] = df[currency_other_col].apply(resolve_currency_other)

    mask_other = df[out_col].eq("OTHER")
    df.loc[mask_other, out_col] = df.loc[mask_other, out_other_clean_col]

    df[out_col] = df[out_col].astype(str).str.strip().str.upper()
    df.loc[~df[out_col].str.match(r"^[A-Z]{3}$", na=False), out_col] = pd.NA

    return df


In [21]:
 df = standardize_currencies(df)

In [22]:
print(df["currency_final"].value_counts(dropna=False).head(30))

currency_final
USD     23486
CAD      1677
GBP      1596
EUR       656
AUD       509
SEK        38
CHF        38
JPY        23
ZAR        18
INR        16
SGD        13
DKK        12
NOK        10
MYR         9
BRL         8
PLN         8
<NA>        7
ILS         7
PHP         7
CZK         7
ARS         5
THB         5
CNY         4
MXN         4
HKD         4
KRW         4
COP         3
IDR         2
TRY         2
NZD         2
Name: count, dtype: int64


In [23]:
df.loc[df["currency"].astype("string").str.upper().eq("OTHER"),
             ["currency", "currency_other", "currency_other_clean", "currency_final"]].drop_duplicates().head(30)

,currency,currency_other,currency_other_clean,currency_final
434,Other,INR,INR,INR
603,Other,Peso Argentino,ARS,ARS
1311,Other,MYR,MYR,MYR
1840,Other,CHF,CHF,CHF
1924,Other,NOK,NOK,NOK
2473,Other,USD,USD,USD
2639,Other,BR$,BRL,BRL
2980,Other,SEK,SEK,SEK
3162,Other,Dkk,DKK,DKK
3261,Other,EUR,EUR,EUR


In [24]:
print(df.loc[df["currency_final"].isna(), ["currency", "currency_other"]].drop_duplicates().head(100))

      currency currency_other
9344     Other            NaN
14915    Other         Equity


In [25]:
df.loc[
    df["currency_other"].astype("string").str.upper() == "EQUITY",
    "currency_final"
] = "NON_MONETARY"

df.loc[df["currency_final"].isna(), "currency_final"] = "UNKNOWN"


In [26]:
print(df.loc[df["currency_final"].isna(), ["currency", "currency_other"]].drop_duplicates().head(100))

Empty DataFrame
Columns: [currency, currency_other]
Index: []


In [27]:
currency_list = sorted(df["currency_final"].dropna().unique())
currency_list = [c for c in currency_list if c not in ["UNKNOWN", "NON_MONETARY"]]
currency_list

['ARS',
 'AUD',
 'BDT',
 'BRL',
 'CAD',
 'CHF',
 'CNY',
 'COP',
 'CZK',
 'DKK',
 'EUR',
 'GBP',
 'HKD',
 'HRK',
 'IDR',
 'ILS',
 'INR',
 'JPY',
 'KRW',
 'LKR',
 'MXN',
 'MYR',
 'NGN',
 'NOK',
 'NZD',
 'PHP',
 'PLN',
 'SAR',
 'SEK',
 'SGD',
 'THB',
 'TRY',
 'TTD',
 'TWD',
 'TZS',
 'USD',
 'ZAR',
 'ZMW']

In [28]:
df.columns

Index(['timestamp', 'age', 'industry', 'job_title', 'job_title_context',
       'annual_salary_raw', 'additional_compensation_raw', 'currency',
       'currency_other', 'income_context', 'country_raw', 'state_us_raw',
       'city_raw', 'experience_years_total', 'experience_years_field',
       'education_level', 'gender', 'race', 'country_norm', 'city_norm',
       'job_title_norm', 'country_clean', 'annual_salary', 'currency_final',
       'currency_other_clean'],
      dtype='object')

In [32]:
df[['currency','currency_other', 'currency_other_clean', 'currency_final', 'annual_salary']].tail(30)

,currency,currency_other,currency_other_clean,currency_final,annual_salary
28161,ZAR,NaN,None,ZAR,5
28162,USD,other,None,USD,0
28163,USD,NaN,None,USD,30000
28164,USD,NaN,None,USD,12000
28165,USD,NaN,None,USD,40000
28166,USD,NaN,None,USD,40000
28167,USD,NaN,None,USD,68000
28168,USD,NaN,None,USD,0
28169,USD,NaN,None,USD,46800
28170,USD,NaN,None,USD,60000


In [35]:
# Función para obtener tasas de cambio a COP usando la API open.er-api.com
def fetch_rates_to_cop(symbols, base_for_api="USD"):

    symbols = [str(s).upper() for s in symbols]

    url = f"https://open.er-api.com/v6/latest/{base_for_api}"
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    data = r.json()

    # Validación básica
    rates = data.get("rates", {})
    if not rates or "COP" not in rates:
        raise ValueError(f"API no devolvió rates válidos o falta COP. Keys: {list(data.keys())}")

    usd_to_cop = rates["COP"]  # USD -> COP según API

    out = {}
    for cur in symbols:
        if cur == "COP":
            out[cur] = 1.0
            continue

        base_to_cur = rates.get(cur)  # USD -> CUR
        if base_to_cur is None or base_to_cur == 0:
            out[cur] = None
        else:
            # CUR -> COP = (USD -> COP) / (USD -> CUR)
            out[cur] = usd_to_cop / base_to_cur

    return out, data


In [36]:

def save_fx_metadata(api_data, rates_dict, path="../docs/fx_metadata.json"):
    metadata = {
        "api_provider": "open.er-api.com",
        "base_currency": api_data.get("base_code") or api_data.get("base"),
        "fx_date": api_data.get("time_last_update_utc") or api_data.get("time_last_update"),
        "retrieved_at": datetime.utcnow().isoformat(),
        "cop_reference_rate": (api_data.get("rates", {}) or {}).get("COP"),
        "converted_rates_to_COP": rates_dict
    }

    path = Path(path)
    path.parent.mkdir(parents=True, exist_ok=True)

    with open(path, "w", encoding="utf-8") as f:
        json.dump(metadata, f, indent=4, ensure_ascii=False)


currency_list = df["currency_final"].dropna().unique().tolist()
rates_to_cop, api_data = fetch_rates_to_cop(currency_list)
save_fx_metadata(api_data, rates_to_cop)


/var/folders/0s/2_dbpwwx125093whj8n7wyf00000gn/T/ipykernel_27717/1616596371.py:6: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "retrieved_at": datetime.utcnow().isoformat(),


In [37]:
# Salary en COP
df["salary_COP"] = (
    df["annual_salary"] * df["currency_final"].map(rates_to_cop)
).round(1)

# Salary en USD (usando la misma referencia USD->COP)
usd_to_cop = rates_to_cop.get("USD")

df["salary_USD"] = (
    df["salary_COP"] / usd_to_cop
).round(1)

In [61]:
df[["annual_salary", "currency_final", "salary_COP", "salary_USD"]].head(30)

,annual_salary,currency_final,salary_COP,salary_USD
0,55000,USD,200217722.3,55000.0
1,54600,GBP,270195648.9,74223.0
2,34000,USD,123770955.6,34000.0
3,62000,USD,225699977.9,62000.0
4,60000,USD,218419333.4,60000.0
5,62000,USD,225699977.9,62000.0
6,33000,USD,120130633.4,33000.0
7,50000,USD,182016111.2,50000.0
8,112000,USD,407716089.1,112000.0
9,45000,USD,163814500.1,45000.0


In [60]:
# Limpiar columna de compensación adicional
df["additional_compensation"] = (
    df["additional_compensation_raw"]
        .astype(str)
        .str.replace(",", "", regex=False)
)

df["additional_compensation"] = (
    pd.to_numeric(df["additional_compensation"], errors="coerce")
    .astype("Int64")
)

In [65]:
# Salary en COP
df["compensation_COP"] = (
    df["additional_compensation"] *
    df["currency_final"].map(rates_to_cop)
).round(1)

# Total compensation en COP (salario + compensación adicional)
df["total_compensation_COP"] = (
    df["salary_COP"].fillna(0) +
    df["compensation_COP"].fillna(0)
).round(1)

In [68]:
df[["annual_salary","additional_compensation", "currency_final", "salary_COP", "salary_USD", "compensation_COP", "total_compensation_COP"]].head(30)

,annual_salary,additional_compensation,currency_final,salary_COP,salary_USD,compensation_COP,total_compensation_COP
0,55000,0,USD,200217722.3,55000.0,0.0,200217722.3
1,54600,4000,GBP,270195648.9,74223.0,19794553.0,289990201.9
2,34000,<NA>,USD,123770955.6,34000.0,<NA>,123770955.6
3,62000,3000,USD,225699977.9,62000.0,10920966.7,236620944.6
4,60000,7000,USD,218419333.4,60000.0,25482255.6,243901589.0
5,62000,<NA>,USD,225699977.9,62000.0,<NA>,225699977.9
6,33000,2000,USD,120130633.4,33000.0,7280644.4,127411277.8
7,50000,<NA>,USD,182016111.2,50000.0,<NA>,182016111.2
8,112000,10000,USD,407716089.1,112000.0,36403222.2,444119311.3
9,45000,0,USD,163814500.1,45000.0,0.0,163814500.1


In [52]:
def clean_city(df, raw_col="city_raw", norm_col="city_norm", out_col="city_clean"):
    
    # Base
    df[out_col] = df[raw_col].astype(str).str.strip()
    s = df[norm_col].fillna("").astype(str).str.strip()

    mask_invalid = (
        s.eq("") |
        s.str.fullmatch(r"\d+", na=False) |             # solo números
        s.str.fullmatch(r"[^\w]+", na=False) |          # solo símbolos
        ~s.str.contains(r"[a-z]", na=False) |           # no tiene letras
        s.str.contains(r"anonymous|can't share|too small|remote|n/?a", na=False) |
        (s.str.split().str.len() >= 6)                  # frases largas
    )

    df.loc[mask_invalid, out_col] = "Unknown"

    # Limpieza final (lo válido)
    mask_valid = df[out_col] != "Unknown"

    df.loc[mask_valid, out_col] = (
        df.loc[mask_valid, out_col]
          .str.replace(r"\s+", " ", regex=True)
          .str.strip()
          .str.title()
    )

    return df

df = clean_city(df)

In [53]:
df['age'].describe()

count     28191
unique        7
top       25-34
freq      12697
Name: age, dtype: object

In [41]:
df['job_title'].describe()

count                 28189
unique                14418
top       Software Engineer
freq                    287
Name: job_title, dtype: object

In [42]:
df['experience_years_total'].describe()

count             28191
unique                8
top       11 - 20 years
freq               9641
Name: experience_years_total, dtype: object

In [43]:
df['experience_years_field'].describe()

count             28191
unique                8
top       11 - 20 years
freq               6546
Name: experience_years_field, dtype: object

In [44]:
df['salary_USD'].describe()

count            28184.0
mean       249627.363674
std      26169572.536064
min                  0.0
25%              53000.0
50%             74159.75
75%             106000.0
max         4391942636.0
Name: salary_USD, dtype: Float64

In [71]:
df.columns

Index(['timestamp', 'age', 'industry', 'job_title', 'job_title_context',
       'annual_salary_raw', 'additional_compensation_raw', 'currency',
       'currency_other', 'income_context', 'country_raw', 'state_us_raw',
       'city_raw', 'experience_years_total', 'experience_years_field',
       'education_level', 'gender', 'race', 'country_norm', 'city_norm',
       'job_title_norm', 'country_clean', 'annual_salary', 'currency_final',
       'currency_other_clean', 'salary_COP', 'salary_USD', 'city_clean',
       'additional_compensation', 'compensation_COP',
       'total_compensation_COP'],
      dtype='object')

In [73]:
df.to_excel("../data/processed/salary_clean_global.xlsx", index=False)
df.to_csv("../data/processed/salary_clean_global.csv", index=False)

df.head()

,timestamp,age,industry,job_title,job_title_context,annual_salary_raw,additional_compensation_raw,currency,currency_other,income_context,...,country_clean,annual_salary,currency_final,currency_other_clean,salary_COP,salary_USD,city_clean,additional_compensation,compensation_COP,total_compensation_COP
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,...,USA,55000,USD,None,200217722.3,55000.0,Boston,0,0.0,200217722.3
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,...,UK,54600,GBP,None,270195648.9,74223.0,Cambridge,4000,19794553.0,289990201.9
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,...,USA,34000,USD,None,123770955.6,34000.0,Chattanooga,<NA>,<NA>,123770955.6
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,"62,000",3000.0,USD,NaN,NaN,...,USA,62000,USD,None,225699977.9,62000.0,Milwaukee,3000,10920966.7,236620944.6
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,"60,000",7000.0,USD,NaN,NaN,...,USA,60000,USD,None,218419333.4,60000.0,Greenville,7000,25482255.6,243901589.0


In [85]:
original_descriptions = dict(
    zip(data_dictionary_v1["new_name"], data_dictionary_v1["original_name"])
)
data_dictionary_v2 = pd.DataFrame({
    "column_name": df.columns,
    "data_type": df.dtypes.astype(str).values
})

source_columns = {
    "country_clean": "country_raw",
    "country_norm": "country_raw",
    "city_norm": "city_raw",
    "job_title_norm": "job_title",
    "annual_salary": "annual_salary_raw",
    "currency_final": "currency, currency_other",
    "salary_USD": "annual_salary, currency_final",
    "salary_COP": "salary_USD",
    "city_clean":  "city_raw, city_norm",
    "currency_other_clean": "currency_other",
    "additional_compensation": "additional_compensation_raw",
    "compensation_COP": "additional_compensation, currency_final",
    "total_compensation_COP": "salary_COP, compensation_COP"
}

transformations = {
    "country_clean": "Regex standardization and phrase filtering",
    "country_norm": "Lowercase + accent removal",
    "city_norm": "Lowercase + accent removal",
    "job_title_norm": "Lowercase + accent removal",
    "annual_salary": "Numeric cast from validated numeric input",
    "currency_final": "Currency resolution logic",
    "currency_other_clean": "Currency resolution logic applied to currency_other",
    "salary_USD": "Converted using FX API rates",
    "salary_COP": "Converted using official TRM (USD to COP)",
    "city_clean":  "rules to mark Unknown and Title Case for valid entries",
    "additional_compensation": "Numeric cast from additional_compensation_raw",
    "compensation_COP": "Converted to COP using currency_final mapped to rates_to_cop",
    "total_compensation_COP": "Sum of salary_COP and compensation_COP (NaN treated as 0 for aggregation)"

}


descriptions_v2 = {
    "country_norm": "Normalized country text used for matching/standardization.",
    "city_norm": "Normalized city text used for matching/standardization.",
    "job_title_norm": "Normalized job title text used for grouping/analysis.",
    "country_clean": "Standardized country category used for analysis and visualization.",
    "currency_other_clean": "Cleaned currency code derived from currency_other.",
    "currency_final": "Final currency code used for FX conversions.",
    "currency_other_clean": "Cleaned currency code derived from currency_other free text input.",
    "annual_salary": "Annual salary as numeric value (in original currency).",
    "salary_USD": "Annual salary converted to USD using FX rates for the run date.",
    "salary_COP": "Annual salary converted to COP using USD to COP TRM for the run date.",
    "city_clean": "Standardized city name with invalid entries marked as 'Unknown'.",
    "additional_compensation": "Additional annual monetary compensation as numeric value (in original currency).",
    "compensation_COP": "Additional annual monetary compensation converted to COP using FX rates for the run date.",
    "total_compensation_COP": "Total annual compensation in COP (salary_COP + compensation_COP)."
    
}

data_dictionary_v2["source_columns"] = data_dictionary_v2["column_name"].map(source_columns)
data_dictionary_v2["transformation"] = data_dictionary_v2["column_name"].map(transformations)
data_dictionary_v2["description"] = (
    data_dictionary_v2["column_name"].map(original_descriptions)
)
data_dictionary_v2["description"] = (
    data_dictionary_v2["column_name"].map(descriptions_v2).combine_first(data_dictionary_v2["description"])
).fillna("")

data_dictionary_v2

,column_name,data_type,source_columns,transformation,description
0,timestamp,object,NaN,NaN,Timestamp
1,age,object,NaN,NaN,How old are you?
2,industry,object,NaN,NaN,What industry do you work in?
3,job_title,object,NaN,NaN,Job title
4,job_title_context,object,NaN,NaN,"If your job title needs additional context, pl..."
5,annual_salary_raw,object,NaN,NaN,What is your annual salary? (You'll indicate t...
6,additional_compensation_raw,float64,NaN,NaN,How much additional monetary compensation do y...
7,currency,object,NaN,NaN,Please indicate the currency
8,currency_other,object,NaN,NaN,"If ""Other,"" please indicate the currency here:"
9,income_context,object,NaN,NaN,"If your income needs additional context, pleas..."


In [86]:
data_dictionary_v2.to_excel("../docs/data_dictionary_v2_modeled.xlsx", index=False)
data_dictionary_v2.to_csv("../docs/data_dictionary_v2_modeled.csv", index=False)

In [76]:
df.head()

,timestamp,age,industry,job_title,job_title_context,annual_salary_raw,additional_compensation_raw,currency,currency_other,income_context,...,country_clean,annual_salary,currency_final,currency_other_clean,salary_COP,salary_USD,city_clean,additional_compensation,compensation_COP,total_compensation_COP
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,...,USA,55000,USD,None,200217722.3,55000.0,Boston,0,0.0,200217722.3
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,...,UK,54600,GBP,None,270195648.9,74223.0,Cambridge,4000,19794553.0,289990201.9
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,...,USA,34000,USD,None,123770955.6,34000.0,Chattanooga,<NA>,<NA>,123770955.6
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,"62,000",3000.0,USD,NaN,NaN,...,USA,62000,USD,None,225699977.9,62000.0,Milwaukee,3000,10920966.7,236620944.6
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,"60,000",7000.0,USD,NaN,NaN,...,USA,60000,USD,None,218419333.4,60000.0,Greenville,7000,25482255.6,243901589.0


In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28191 entries, 0 to 28190
Data columns (total 31 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   timestamp                    28191 non-null  object 
 1   age                          28191 non-null  object 
 2   industry                     28110 non-null  object 
 3   job_title                    28189 non-null  object 
 4   job_title_context            7289 non-null   object 
 5   annual_salary_raw            28191 non-null  object 
 6   additional_compensation_raw  20835 non-null  float64
 7   currency                     28191 non-null  object 
 8   currency_other               220 non-null    object 
 9   income_context               3051 non-null   object 
 10  country_raw                  28191 non-null  object 
 11  state_us_raw                 23128 non-null  object 
 12  city_raw                     28109 non-null  object 
 13  experience_years